In [ ]:
#!jupyter nbconvert --to=python Chlsurf_OCCCI_Comparison_python3.ipynb

In [ ]:
class Chlsurf_OCCCI_comp:
    '''
    class Chlsurf_OCCCI_comp(runname,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.matfileChlsurf=matfileChlsurf
        

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle
        import scipy.io as spio


        import pyfesom2 as pf
        
        from plot_Taylor_normalized import plt_Taylor_norm
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = [self.fyear, self.lyear]
        
        # load FESOM CHl.a data -------------------------------------------------------------------------------------
        ncFESOMChl = resultpath + '/ChlA.mo1.1948.nc'
        ncFESOMDiaChl = resultpath + '/DiaChl.mo1.1948.nc'
        #!ncdump -h $ncFESOMChl
        #!ncdump -h $ncFESOMDiaChl
        
        ChlAfesom = pf.get_data(resultpath, "ChlA", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        DiaChlfesom = pf.get_data(resultpath, "DiaChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        ChlAfesom_surf = ChlAfesom[:,0]
        ChlAfesom_mean = np.mean(ChlAfesom, axis=1)
        DiaChlfesom_mean = np.mean(DiaChlfesom, axis=1)

        labelfesom = 'FESOM Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        labelfesomDia = 'FESOM Diatom Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        #unitfesom = 'Chl.a [mmol m$^{-3}$]' 
        
        # convert FESOM CHl.a data -------------------------------------------------------------------------------------
        #Chl.a : 893.509 g/mol
        if True:
            conv = 893.509
            ChlAfesom_surf = np.log10(ChlAfesom_surf * conv)
            ChlAfesom_mean = np.log10(ChlAfesom_mean * conv)
            DiaChlfesom_mean = np.log10(DiaChlfesom_mean * conv)
            unitfesom = 'log$_{10}$(Chl.a) [mg m$^{-3}$]'
        
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        matChl = spio.loadmat(self.matfileChlsurf, squeeze_me=True)
        
        annualchl   = np.log10(matChl['x'])
                
        lat         = np.arange(-89.5,90.,1.)
        lon         = np.arange(-179.5,180.,1.)

        latdic, londic = np.meshgrid(lat, lon)
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2

        # interpolate OCCCI CHl.a data -------------------------------------------------------------------------------------
        OCCCIchla = griddata((londic.ravel(), latdic.ravel()), annualchl.ravel(), (lon_fesom, lat_fesom), method='nearest')
        OCCCIchla = np.ma.filled(OCCCIchla, np.nan)
        
        OCCCIlabel = 'OCCCI: Surface Chl concentration (mean over depth??)'
        OCCCIunit = 'log$_{10}$(Chlorophyll) [mg Chl m$^{-3}$]'
        
        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        OCCCIchla_ma = np.copy(OCCCIchla)
        OCCCIchla_ma[ChlAfesom_mean == 0] = 0
            
        
        # plotting  -------------------------------------------------------------------------------------

        print('\nPlotting log10(Chl.a)\nSurface\nFESOM min = {4}, max = {5}\nMean over depth:\nFESOM min = {0}, max = {1}\nOCCCI min = {2}, max = {3}'.format(
                np.nanmin(ChlAfesom_mean),np.nanmax(ChlAfesom_mean),
                np.nanmin(OCCCIchla_ma),np.nanmax(OCCCIchla_ma),
                np.nanmin(ChlAfesom_surf),np.nanmax(ChlAfesom_surf)))

        fig_surface = pf.plot(mesh, [ChlAfesom_surf], 
                rowscol= (1,1),
                levels = (-3,4,71),
                units=unitfesom, 
                mapproj=self.mapproj, # robinson projection takes more time!
                titles=['Surface '+labelfesom],
                figsize = (10,10))
        
        fig_data = pf.plot(mesh, [OCCCIchla_ma,
                                  #ChlAfesom_surf,
                                  ChlAfesom_mean, 
                                  OCCCIchla_ma-ChlAfesom_mean
                                 ], 
                rowscol= (3,1),
                levels = (-3,4,71),
                units=OCCCIunit, 
                mapproj=self.mapproj, # robinson projection takes more time!
                titles=[OCCCIlabel,
                        #'Surface '+labelfesom, 
                        'Mean over depth '+labelfesom, 
                        'Initialization fields minus FESOM (mean over depth)'],
                figsize = (20,20))
               
        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):
            plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False)  

        # statistics  -------------------------------------------------------------------------------------            
        # preparation of datasets
        if np.isnan(np.min(OCCCIchla_ma)): print('WARNING: The interpolated OCCCI field contains NaNs at depth')
        if np.isnan(np.min(ChlAfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

        # get statistics only from valid OCCCI gridpoints 
        ind_stat = np.where(np.isfinite(OCCCIchla_ma))
        
        title = 'Taylor Diagram for surface Chlorophyll\n(mean over depth)'
        plt_Taylor_norm(OCCCIchla_ma[ind_stat],ChlAfesom_mean[ind_stat],mask=True,title=title)

        if False: #old
            taylor_stats1 = sm.taylor_statistics(OCCCIchla_ma[ind_stat],ChlAfesom_mean[ind_stat])
            sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
            crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
            ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

            label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

            fig_stat = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
            sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                  titleOBS = 'observation', markerLabel = label,
                                  markerLabelColor = 'c',
                                  markerColor = 'c', markerLegend = 'on',
                                  #tickRMS = range(0,5,1), 
                                  tickRMSangle = 135.0,
                                  colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                  titleRMS = 'off', #tickSTD = range(0,10,2 ),
                                  #axismax = 10.0, 
                                  colSTD = 'b', styleSTD = '-.',
                                  widthSTD = 1.0, titleSTD = 'on',
                                  colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                  titleCOR = 'off')
            plt.title('Taylor Diagram for surface Chlorophyll\n(mean over depth)', loc='right')


        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):                
            plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False) 
        

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'mo1'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + runid
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    htmlname     =  runid + '.html'
    htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    #modulepath = '/home/ollie/mozeisin/py_recom/GlobalAssessment'
    #modulepath   = '/home/ollie/jhauck/py_fesom/modules/'

    #obspath      = '/home/ollie/mseifert/py_fesom/CreateRunFiles/Initialization/Output/'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1948
    last_year  = 1950

    # OCCCI ------------------------------------------------------------------------------------  
    matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'
    
    # now test:
    test = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                              layerwise=False,savefig=False)